In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
# load datasets into data frames raw and working
df_imdb_raw = pd.read_csv("imdb_videogames.csv")
df_wiki_raw = pd.read_csv("wiki_list.csv")

In [24]:
df_imdb = df_imdb_raw.copy()
df_wiki = df_wiki_raw.copy()

In [25]:
# standardising column names
df_imdb.columns = (
    df_imdb.columns
      .str.lower()
      .str.strip()
      .str.replace(" ", "_")
)

In [26]:
df_wiki.columns = (
    df_wiki.columns
      .str.lower()
      .str.strip()
      .str.replace(" ", "_")
)

In [39]:
df_wiki = df_wiki.rename(columns={
    "title": "original_title",
    "title.1": "remaster_title",
    "release_year" : "original_release_year",
    "release_year.1" : "remaster release year",
    "platform(s)" : "original_platform",
    "platform(s).1" : "remaster_platform"    
    
})

In [28]:
# cleaning title names function
def clean_title(title):
    title = title.lower().strip()
    # Remove trademark symbols
    title = re.sub(r'™|®', '', title)
    # Replace colon, dash, HD, remastered, remake with a SPACE
    title = re.sub(r':| - | hd| remastered| remake', ' ', title)
    # Remove anything in parentheses
    title = re.sub(r'\(.*?\)', '', title)
    # Remove all other non-alphanumeric characters but keep spaces
    title = re.sub(r'[^a-z0-9 ]', '', title)
    # Replace multiple spaces with a single space
    title = re.sub(r'\s+', ' ', title)
    return title.strip()

In [30]:
# create cleaned titles using clean_title function
df_imdb["clean_title"] = df_imdb["name"].apply(clean_title)
df_wiki["clean_title_og"] = df_wiki["original_title"].apply(clean_title)
df_wiki["clean_title_rm"] = df_wiki["remaster_title"].apply(clean_title)

In [31]:
# splitting imdb data into original and remasters/remakes
remaster_keywords = ["remaster", "remastered", "remake", "hd", "anniversary", "definitive", "collection"]

In [32]:
# converting year column from float to int64 (supports null values) 
df_imdb["year"] = pd.to_numeric(df_imdb["year"], errors="coerce").astype("Int64")

In [33]:
# checking for NaN and empty strings in cleaned titles for imdb_remaster and imdb_normal_games
df_imdb[df_imdb["clean_title"].isna() | (df_imdb["clean_title"] == "")]

,name,url,year,certificate,rating,votes,plot,action,adventure,comedy,crime,family,fantasy,mystery,sci-fi,thriller,clean_title
17494,>=<,https://www.imdb.com/title/tt10345886/?ref_=ad...,2010,NaN,NaN,NaN,Add a Plot,False,False,False,False,True,False,False,False,False,


In [63]:
df_imdb["votes"] = (
    df_imdb["votes"]
    .astype(str)
    .str.replace(",", "", regex=False)
)

In [73]:
df_imdb["votes"] = pd.to_numeric(df_imdb["votes"], errors="coerce").astype("Int64")

In [68]:
# function to add column to dataframe with either true or false if clean title has any of the remaster keywords in it (using regular expressions)
pattern = r"\b(?:{})\b".format("|".join(remaster_keywords))

df_imdb["is_remaster"] = (
    df_imdb["clean_title"]
    .str.contains(pattern, regex=True, na=False))

In [69]:
# partioning imdb dataset into seperate dataframes for remasters/remakes and non remastered games
imdb_normal_games = df_imdb[df_imdb["is_remaster"] == False]
imdb_remasters = df_imdb[df_imdb["is_remaster"] == True]

In [ ]:
imdb_normal_games.info()

In [ ]:
# randomly selecting records (spot check)
df_imdb.sample(20)

In [113]:
# creating column with unique imdb url text identifier
df_imdb["imdb_id"] = df_imdb["url"].str.extract(r"(tt\d+)")

In [122]:
# deduplicating for imdb entries with same ids in url
df_imdb = (
    df_imdb.sort_values("votes", ascending=False)
      .drop_duplicates(subset=["imdb_id"], keep="first")
)

In [123]:
# validating
df_imdb["imdb_id"].is_unique

True

In [127]:
# rechecking for clean_title and year duplicates
df_imdb["clean_title"].value_counts().loc[lambda x: x > 1].head(10)

clean_title
star trek                         6
batman                            5
spiderman                         5
spiderman 2                       5
the terminator                    5
shadowrun                         4
jurassic park                     4
monopoly                          4
star wars                         4
the adventures of batman robin    4
Name: count, dtype: int64

In [132]:
# spot checking duplicates again after removing imdb id duplicates
df_imdb.loc[
    (df_imdb["clean_title"] == "star trek"),
    ["name", "year", "votes", "rating", "imdb_id", "url"]
]

,name,year,votes,rating,imdb_id,url
787,Star Trek,2013,946,6.9,tt2191526,https://www.imdb.com/title/tt2191526/?ref_=adv...
20000,Star Trek,1991,115,6.9,tt0152592,https://www.imdb.com/title/tt0152592/?ref_=adv...
19671,Star Trek,1983,51,7.5,tt0475733,https://www.imdb.com/title/tt0475733/?ref_=adv...
19570,Star Trek,1971,28,6.9,tt1982844,https://www.imdb.com/title/tt1982844/?ref_=adv...
5003,Star Trek,1972,15,5.4,tt1982161,https://www.imdb.com/title/tt1982161/?ref_=adv...
20593,Star Trek,1991,<NA>,NaN,tt14545272,https://www.imdb.com/title/tt14545272/?ref_=ad...


In [133]:
# checking how many entries with same titles and year
df_imdb.duplicated(subset=["clean_title", "year"]).sum()

64